In [1]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import re
import time
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/My Drive/iss/isy5004_project/')
!ls

Mounted at /content/gdrive
'Augmentation Project (1).ipynb'
'Augmentation Project (2).ipynb'
'Augmentation Project.ipynb'
 augmented_image2
'background removal.ipynb'
 best_inception_model.h5
 CNN-Model-Distance.ipynb
'CNN Modeling.ipynb'
'Copy of Keras_Feature_Extractor_MainModel.ipynb'
 data
'Detectron2 YS.ipynb'
 final_augmentated_production
 final_augmented_production
 hipvan_image
 ikea_image
'ikea scrapper.ipynb'
 image_search2.ipynb
 input.jpg
 Keras_Extracting_For_Model.ipynb
 Keras_Feature_Extractor.ipynb
 Keras_Feature_Extractor_MainModel.ipynb
 Mask_RCNN
 mask_rcnn_coco.h5
 models
 object_detection_classes_yolov3.txt
'Object Detection with BB and object counting.ipynb'
'Object Detection with BB.ipynb'
 production_ready_code
 telegram_bot.ipynb
 temp
 test_df_ikea_hipvan.csv
 train_df_ikea_hipvan.csv
 trainset
 trainset2
'Unsuccessful Background Removal.ipynb'
 Untitled0.ipynb
 utils.py
 web_scraper.ipynb
'work breakdown.gsheet'
 yolov3.cfg
 yolov3.weights


Scrap Ikea website:

In [3]:
source = urllib.request.urlopen('https://www.ikea.com/sg/en/cat/products-products/').read()
soup = bs.BeautifulSoup(source,'lxml')
all_link = []
for _ in soup.find_all("a", ["vn-link vn-nav__link"]):
  all_link.append(re.search(r'.*-(.*)\/$', _['href']).group(1))
len(all_link)

def get_data(category):

  payload = {'sessionId': '13f8db09-01e9-44e4-ac00-b58b83718a47', 
            'category': category,
            'sort': 'RELEVANCE',
            'size': '10000',
            'c': 'lf',
            'v': '20210219'}

  url = 'https://sik.search.blue.cdtapps.com/sg/en/product-list-page'

  r = requests.get(url, params=payload)
  return r

all_data = pd.DataFrame()

for _ in set(all_link):
  data = get_data(_)
  if data.status_code == 200:
    temp = pd.DataFrame(data.json()['productListPage']['productWindow'])
    temp['category_code'] = _
    temp['category_name'] = data.json()['productListPage']['category']['name']
    all_data = all_data.append(temp)
    print("successfully retrieved data for category_code: {}, category_name: {}".format(_, data.json()['productListPage']['category']['name']))
  else:
    print("error in getting data category_code: {}, status code: {}".format(_, data(_).status_code))

successfully retrieved data for category_code: 36812, category_name: Smart lighting & improvement
successfully retrieved data for category_code: 46080, category_name: Room dividers
successfully retrieved data for category_code: 24822, category_name: Bed legs
successfully retrieved data for category_code: fu005, category_name: Trolleys & kitchen islands
successfully retrieved data for category_code: 20494, category_name: Plant stands & movers
successfully retrieved data for category_code: 19141, category_name: Café furniture
successfully retrieved data for category_code: tl002, category_name: Curtains, curtain rods & blinds
successfully retrieved data for category_code: fu004, category_name: Tables & desks
successfully retrieved data for category_code: 16298, category_name: Door handles & knobs
successfully retrieved data for category_code: 18690, category_name: Baby textiles
successfully retrieved data for category_code: 34204, category_name: Outdoor rugs
successfully retrieved data fo

In [ ]:
all_data.to_csv('data/ikea_products.csv')

In [ ]:
all_data = pd.read_csv('data/ikea_products.csv', index_col=False)

In [ ]:
all_data.head()

,Unnamed: 0,name,typeName,itemMeasureReferenceText,mainImageUrl,pipUrl,id,itemNoGlobal,metadata,onlineSellable,lastChance,gprDescription,colors,priceNumeral,currencyCode,category_code,category_name
0,0,LÄTT,Children's table with 2 chairs,NaN,https://www.ikea.com/sg/en/images/products/lae...,https://www.ikea.com/sg/en/p/laett-childrens-t...,10178413,50178411,Relevance PLP,True,False,"{'numberOfVariants': 0, 'variants': []}","[{'name': 'beige', 'id': '10003', 'hex': 'f4f3...",29.9,SGD,18767,Children's small furniture
1,1,FLISAT,Children's table,83x58 cm,https://www.ikea.com/sg/en/images/products/fli...,https://www.ikea.com/sg/en/p/flisat-childrens-...,30298419,50298418,Relevance PLP,True,False,"{'numberOfVariants': 0, 'variants': []}","[{'name': 'beige', 'id': '10003', 'hex': 'f4f3...",89.0,SGD,18767,Children's small furniture
2,2,DUNDRA,Activity table with storage,NaN,https://www.ikea.com/sg/en/images/products/dun...,https://www.ikea.com/sg/en/p/dundra-activity-t...,80472500,30472499,Relevance PLP,True,False,"{'numberOfVariants': 0, 'variants': []}","[{'name': 'white', 'id': '10156', 'hex': 'ffff...",159.0,SGD,18767,Children's small furniture
3,3,SUNDVIK,Children's table,76x50 cm,https://www.ikea.com/sg/en/images/products/sun...,https://www.ikea.com/sg/en/p/sundvik-childrens...,90201674,10201673,Relevance PLP,True,False,"{'numberOfVariants': 0, 'variants': []}","[{'name': 'white', 'id': '10156', 'hex': 'ffff...",59.0,SGD,18767,Children's small furniture
4,4,SLÄKT,"Pouffe/mattress, foldable",NaN,https://www.ikea.com/sg/en/images/products/sla...,https://www.ikea.com/sg/en/p/slaekt-pouffe-mat...,90362964,10362963,Relevance PLP,True,False,"{'numberOfVariants': 0, 'variants': []}","[{'name': 'grey', 'id': '10028', 'hex': '94949...",90.0,SGD,18767,Children's small furniture


In [ ]:
all_data[all_data.id == "40446136"].category_name #JÄRNSPARV_Handle_40446136.jpg

855                      Children
3958     Storage solution systems
10248                Baby product
Name: category_name, dtype: object

In [ ]:
folder = 'ikea_image'
img_count = 0
for i in tqdm(range(len(all_data))):
  retry_second = 0
  path = folder + '/' + all_data.category_name.iloc[i] + '/'
  fullpath = path + all_data.name.iloc[i].replace('/','|') + '_' + all_data.iloc[i].typeName.replace('/','|') + '_' + all_data.id.iloc[i] + '.jpg'

  if not os.path.exists(path):
    os.makedirs(path)

  if os.path.isfile(fullpath) == 0 and pd.isna(all_data.iloc[i].mainImageUrl) == 0:
    for attempt in range(5):

      try:
        urllib.request.urlretrieve(all_data.mainImageUrl.iloc[i], fullpath)
      except:
        print('try: {} error in retrieving row {}, URL: {}'.format(attempt, i, all_data.mainImageUrl.iloc[i]))
        retry_second += 1
        time.sleep(retry_second)

      else:
        img_count += 1
        break
      
  else:
    img_count += 1

print('{} image downloaded'.format(img_count))

  2%|▏         | 189/12418 [00:07<07:59, 25.52it/s]   


KeyboardInterrupt: ignored

Scrap Hipvan website:

In [ ]:
source = urllib.request.urlopen('https://www.hipvan.com/').read()
soup = bs.BeautifulSoup(source,'lxml')
all_link = []

# regex = re.compile(r'(?<=href).*')
for _ in soup.find_all("a", ["c-nav__menu__cats-link js-nav__menu__cats-link"]):
  # print(_)
  all_link.append(re.search(r'(?<=href="/).*', str(_))[0].split('?')[0])
# len(all_link)



In [ ]:
all_link[:10]

['new-arrivals-2/adapted-scandinavian',
 'new-arrivals-2/japanese-minimalist',
 'new-arrivals-2/modern-contemporary',
 'new-arrivals-2/industrial',
 'new-arrivals-2/bohemian-resort-living',
 'new-arrivals-2/botanical-accents',
 'new-arrivals-2/rose-gold-champagne',
 'new-arrivals-2/carson-by-hipvan',
 'new-arrivals-2/cadencia-by-hipvan',
 'new-arrivals-2/gianna-by-hipvan']

In [ ]:
hipvan.keys()#['data']

dict_keys(['success', 'status', 'data'])

In [ ]:
hipvan['status']#['data']['products']

200

In [ ]:
import json

def get_data(item_type):

  payload = {"query":{"handle":item_type},
             "sort":"featured_asc",
             "filter":"","page":1,
             "per_page":"48",
             "platform":"desktop-web",
             }

  body = json.dumps(payload)
  
  headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

  url = 'https://api-production.hipvan.com/api/v1/search/collections'

  r = requests.post(url, json=payload, headers=headers)
  return r

In [ ]:
all_data = pd.DataFrame()

for _ in tqdm(all_link):
  category = _.split('/')[0]
  item_type = _.split('/')[1]
  if category not in ['new-arrivals-2', 'clearance']:
    data = get_data(item_type)
    if data['status'] == 200:
      temp = pd.DataFrame(data['data']['products'])
      temp['category'] = category
      all_data = all_data.append(temp)
      print("successfully retrieved data for category: {}, category_name: {}".format(category, item_type))
    else:
      print("error in getting data category_code: {}, status code: {}".format(_, data['status']))

 14%|█▍        | 36/258 [00:03<00:20, 10.75it/s]

successfully retrieved data for category: sofas, category_name: sofas-loveseats


 14%|█▍        | 37/258 [00:06<03:11,  1.15it/s]

successfully retrieved data for category: sofas, category_name: l-shaped-sofas


 15%|█▍        | 38/258 [00:07<04:21,  1.19s/it]

...

successfully retrieved data for category: kids, category_name: kids-food-storage


 93%|█████████▎| 240/258 [07:42<00:36,  2.03s/it]

successfully retrieved data for category: kids, category_name: kids-backpacks


100%|██████████| 258/258 [07:45<00:00,  1.80s/it]

successfully retrieved data for category: kids, category_name: kids-toys


In [ ]:
all_data.to_csv('data/hipvan_products.csv')

In [ ]:
all_data = pd.read_csv('data/hipvan_products.csv', index_col=False)

In [ ]:
all_data.sort_values(by='id')

,Unnamed: 0,id,fast_shipping,sold_out,sale_off,product_properties,handle,brand_name,tags,name,colour_group,show_alternative_colours,shipping_duration,show_on_collection,free_delivery,hiptruck_id,vendor_name,relative_url,product_variants_ids,taxon_hiptruck_ids,compare_at_price_of_product_variant,product_variant_default_id,min_price_of_product_variant,colour_group_products,product_variants,min_price_of_product_variant_with_format,compare_at_price_of_product_variant_with_format,images,category
6465,17,908,False,False,0.0,"{'colour': 'White', 'overall_dimensions_text':...",animal-parade-seasoning-shaker,Qualy,"['condimentsets', '14daysreturn', 'gifting']",Animal Parade Seasoning Shaker,NaN,False,6 working days,True,False,908,Qualy,products/animal-parade-seasoning-shaker,[1335],"[5363, 9446, 2710, 2646, 9749, 8036, 1263]",49.9,1335,49.9,[],"[{'id': '1335', 'shipping_date': '19 May 2021'...",$49.90,$49.90,['https://hipvan-images-production.imgix.net/p...,lifestyle-all
4592,6,908,False,False,0.0,"{'colour': 'White', 'overall_dimensions_text':...",animal-parade-seasoning-shaker,Qualy,"['condimentsets', '14daysreturn', 'gifting']",Animal Parade Seasoning Shaker,NaN,False,6 working days,True,False,908,Qualy,products/animal-parade-seasoning-shaker,[1335],"[5363, 9446, 2710, 2646, 9749, 8036, 1263]",49.9,1335,49.9,[],"[{'id': '1335', 'shipping_date': '19 May 2021'...",$49.90,$49.90,['https://hipvan-images-production.imgix.net/p...,kitchen
4675,41,1158,False,False,10.0,"{'colour': 'Clear', 'overall_dimensions_text':...",assam-double-wall-glass-0-25l-set-of-2,BODUM®,"['glasswareshooters', 'mugsdrinkingglasses']",ASSAM Double Wall Glass 0.25L (Set of 2),NaN,False,6 working days,True,False,1158,BODUM®,products/assam-double-wall-glass-0-25l-set-of-2,[1675],"[4904, 9446, 9309, 3399, 5329]",44.8,1675,39.8,[],"[{'id': '1675', 'shipping_date': '19 May 2021'...",$39.80,$44.80,['https://hipvan-images-production.imgix.net/p...,kitchen
4676,42,1159,False,False,25.0,"{'colour': 'Clear', 'overall_dimensions_text':...",assam-double-wall-glass-0-2l-set-of-2,BODUM®,"['finalsale', 'glasswareshooters', 'mugsdrinki...",ASSAM Double Wall Glass 0.2L (Set of 2),NaN,False,6 working days,True,False,1159,BODUM®,products/assam-double-wall-glass-0-2l-set-of-2,[1676],"[5329, 3399, 4904, 9446, 9309]",39.8,1676,29.8,[],"[{'id': '1676', 'shipping_date': '19 May 2021'...",$29.80,$39.80,['https://hipvan-images-production.imgix.net/p...,kitchen
6456,8,4766,False,False,30.0,"{'colour': None, 'overall_dimensions_text': 'W...",cupcake-kit,Lékué,"['bakingpansdishes', 'warranty', 'gifting']",Cupcake Kit,NaN,False,5 working days,True,False,4766,Lekue,products/cupcake-kit,[6767],"[2710, 9338, 5365, 9446, 2646, 9749, 4453, 6712]",59.0,6767,42.0,[],"[{'id': '6767', 'shipping_date': '18 May 2021'...",$42,$59,['https://hipvan-images-production.imgix.net/p...,lifestyle-all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,7,102756,True,False,0.0,"{'colour': None, 'overall_dimensions_text': ''}",rinco-bonington-classic-bolster-2-types,Rinco Bonington,['pillowsbolsters'],Rinco Bonington Classic Bolster (2 Types),NaN,False,3 working days,True,False,102756,Bedroom Affairs,products/rinco-bonington-classic-bolster-2-types,"[143598, 143597]","[9767, 9768, 9757, 7414, 9756, 9442, 5344, 944...",29.9,143598,29.9,[],"[{'id': '143597', 'shipping_date': '14 May 202...",$29.90,$29.90,['https://hipvan-images-production.imgix.net/p...,soft-furnishings
1103,46,102756,True,False,0.0,"{'colour': None, 'overall_dimensions_text': ''}",rinco-bonington-classic-bolster-2-types,Rinco Bonington,['pillowsbolsters'],Rinco Bonington Classic Bolster (2 Types),NaN,False,3 working days,True,False,102756,Bedroom Affairs,products/rinco-bonington-classic-bolster-2-types,"[143598, 143597]","[9767, 9768, 9757, 7414, 9756, 9442, 5344, 944...",29.9,143598,29.9,[],"[{'id': '143597', 'shipping_date': '14 May 202...",$29.90,$29.90,['https://hipvan-image

In [ ]:
all_data.iloc[10].images[1:-1].split(',')#[1].replace("'","")

["'https://hipvan-images-production.imgix.net/product-images/60494945-4ef1-4625-9f40-b2856a330b36/Premium-Sofas-by-HipVan--Nolan-L-Shaped-Sofa--Slate-(Fabric)-11.png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center'",
 " 'https://hipvan-images-production.imgix.net/product-images/fcf68c23-c6e9-4beb-a844-9dbb22180b2a/Premium-Sofas-by-HipVan--Nolan-L-Shaped-Sofa--Slate-(Fabric)-8.png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center'"]

In [ ]:
folder = 'hipvan_image'
img_count = 0
for i in tqdm(range(all_data.shape[0])):
  retry_second = 0
  links = all_data.iloc[i].images[1:-1].split(',')#[1].replace("'","")
  for j, link in enumerate(links):
    path = folder + '/' + all_data.category.iloc[i] + '/'
    fullpath = path + 'hipvan_' + str(all_data.id.iloc[i]) + '_' + str(j) + '.jpg'

    if not os.path.exists(path):
      os.makedirs(path)

    if os.path.isfile(fullpath) == 0:
      for attempt in range(5):

        try:
          urllib.request.urlretrieve(link.replace("'",""), fullpath)
        except:
          print('try: {} error in retrieving row {}, URL: {}'.format(attempt, i, link.replace("'","")))
          retry_second += 1
          time.sleep(retry_second)

        else:
          img_count += 1
          break
        
    else:
      img_count += 1

print('{} image downloaded'.format(img_count))



  0%|          | 0/7218 [00:00<?, ?it/s]

  1%|▏         | 93/7218 [00:00<00:07, 921.69it/s]

  3%|▎         | 182/7218 [00:00<00:07, 910.84it/s]

  4%|▎         | 264/7218 [00:00<00:07, 878.78it/s]

  5%|▍         | 353/7218 [00:00<00:07, 881.96it/s]

  6%|▌         | 436/7218 [00:00<00:07, 864.53it/s]

  7%|▋         | 528/7218 [00:00<00:07, 878.52it/s]

  8%|▊         | 610/7218 [00:00<00:07, 854.79it/s]

 10%|▉         | 692/7218 [00:00<00:07, 842.93it/s]

 11%|█         | 774/7218 [00:00<00:07, 835.44it/s]

 12%|█▏        | 876/7218 [00:01<00:07, 882.65it/s]

 13%|█▎        | 963/7218 [00:01<00:07, 872.86it/s]

 15%|█▍        | 1050/7218 [00:01<00:07, 855.10it/s]

 16%|█▌        | 1141/7218 [00:01<00:06, 870.36it/s]

 17%|█▋        | 1235/7218 [00:01<00:06, 888.48it/s]

try: 0 error in retrieving row 1284, URL:  https://hipvan-images-production.imgix.net/product-images/1213d900-f2a5-4559-b1e9-37f51f04046c/LH-Inez-BED-SET-(-SS-
try: 1 error in retrieving row 1284, URL:  https://hipvan-images-production.imgix.net/product-images/1213d900-f2a5-4559-b1e9-37f51f04046c/LH-Inez-BED-SET-(-SS-
try: 2 error in retrieving row 1284, URL:  https://hipvan-images-production.imgix.net/product-images/1213d900-f2a5-4559-b1e9-37f51f04046c/LH-Inez-BED-SET-(-SS-
try: 3 error in retrieving row 1284, URL:  https://hipvan-images-production.imgix.net/product-images/1213d900-f2a5-4559-b1e9-37f51f04046c/LH-Inez-BED-SET-(-SS-
try: 4 error in retrieving row 1284, URL:  https://hipvan-images-production.imgix.net/product-images/1213d900-f2a5-4559-b1e9-37f51f04046c/LH-Inez-BED-SET-(-SS-
try: 0 error in retrieving row 1284, URL: -Q-




 17%|█▋        | 1235/7218 [00:19<00:06, 888.48it/s]

try: 1 error in retrieving row 1284, URL: -Q-
try: 2 error in retrieving row 1284, URL: -Q-
try: 3 error in retrieving row 1284, URL: -Q-
try: 4 error in retrieving row 1284, URL: -Q-
try: 0 error in retrieving row 1284, URL: -K-).png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center
try: 1 error in retrieving row 1284, URL: -K-).png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center
try: 2 error in retrieving row 1284, URL: -K-).png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center
try: 3 error in retrieving row 1284, URL: -K-).png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center
try: 4 error in retrieving row 1284, URL: -K-).png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center




 18%|█▊        | 1285/7218 [02:02<1:11:58,  1.37it/s]

 18%|█▊        | 1286/7218 [02:02<57:49,  1.71it/s]  

 18%|█▊        | 1287/7218 [02:03<1:06:28,  1.49it/s]

 18%|█▊        | 1288/7218 [02:03<50:19,  1.96it/s]  

 18%|█▊        | 1289/7218 [02:03<38:38,  2.56it/s]

 18%|█▊        | 1290/7218 [02:04<50:13,  1.97it/s]

 18%|█▊        | 1291/7218 [02:05<1:10:40,  1.40it/s]

 18%|█▊        | 1292/7218 [02:06<52:31,  1.88it/s]  

 18%|█▊        | 1293/7218 [02:06<44:45,  2.21it/s]

 18%|█▊        | 1294/7218 [02:08<1:22:51,  1.19it/s]

 18%|█▊        | 1295/7218 [02:08<1:07:43,  1.46it/s]

...

try: 0 error in retrieving row 4154, URL: https://hipvan-images-production.imgix.net/product-images/4b363641-7bd0-44b5-af18-468708a9506f/Tesora_Marble_Box-White
try: 1 error in retrieving row 4154, URL: https://hipvan-images-production.imgix.net/product-images/4b363641-7bd0-44b5-af18-468708a9506f/Tesora_Marble_Box-White
try: 2 error in retrieving row 4154, URL: https://hipvan-images-production.imgix.net/product-images/4b363641-7bd0-44b5-af18-468708a9506f/Tesora_Marble_Box-White
try: 3 error in retrieving row 4154, URL: https://hipvan-images-production.imgix.net/product-images/4b363641-7bd0-44b5-af18-468708a9506f/Tesora_Marble_Box-White
try: 4 error in retrieving row 4154, URL: https://hipvan-images-production.imgix.net/product-images/4b363641-7bd0-44b5-af18-468708a9506f/Tesora_Marble_Box-White
try: 0 error in retrieving row 4154, URL: _Nickel_02.png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20center
try: 1 error in retrieving row 4154, URL: _Nick



 58%|█████▊    | 4155/7218 [36:09<54:18:07, 63.82s/it]

try: 0 error in retrieving row 4155, URL: https://hipvan-images-production.imgix.net/product-images/70373a1f-388f-4caa-94a1-8f3bdec0cb40/Tesora_Marble_Jewelry_Stand-White
try: 1 error in retrieving row 4155, URL: https://hipvan-images-production.imgix.net/product-images/70373a1f-388f-4caa-94a1-8f3bdec0cb40/Tesora_Marble_Jewelry_Stand-White
try: 2 error in retrieving row 4155, URL: https://hipvan-images-production.imgix.net/product-images/70373a1f-388f-4caa-94a1-8f3bdec0cb40/Tesora_Marble_Jewelry_Stand-White
try: 3 error in retrieving row 4155, URL: https://hipvan-images-production.imgix.net/product-images/70373a1f-388f-4caa-94a1-8f3bdec0cb40/Tesora_Marble_Jewelry_Stand-White
try: 4 error in retrieving row 4155, URL: https://hipvan-images-production.imgix.net/product-images/70373a1f-388f-4caa-94a1-8f3bdec0cb40/Tesora_Marble_Jewelry_Stand-White
try: 0 error in retrieving row 4155, URL: _Nickel_01.png?w=400&h=400&fit=fill&bg=ffffff&auto=format&cs=srgb&markscale=100&markalign=bottom%20cent

Streaming output truncated to the last 5000 lines.

 60%|██████    | 4354/7218 [41:00<23:55,  2.00it/s]

 60%|██████    | 4355/7218 [41:01<24:46,  1.93it/s]

 60%|██████    | 4356/7218 [41:01<22:45,  2.10it/s]

 60%|██████    | 4357/7218 [41:02<24:11,  1.97it/s]

 60%|██████    | 4358/7218 [41:02<25:35,  1.86it/s]

 60%|██████    | 4359/7218 [41:03<30:10,  1.58it/s]

 60%|██████    | 4360/7218 [41:04<30:04,  1.58it/s]

 60%|██████    | 4361/7218 [41:04<29:05,  1.64it/s]

 60%|██████    | 4362/7218 [41:04<22:03,  2.16it/s]

 60%|██████    | 4363/7218 [41:05<22:23,  2.12it/s]

 60%|██████    | 4364/7218 [41:05<19:00,  2.50it/s]

 60%|██████    | 4365/7218 [41:06<20:04,  2.37it/s]

 60%|██████    | 4366/7218 [41:06<25:41,  1.85it/s]

 61%|██████    | 4367/7218 [41:07<28:38,  1.66it/s]

 61%|██████    | 4368/7218 [41:08<28:13,  1.68it/s]

 61%|██████    | 4369/7218 [41:09<33:42,  1.41it/s]

 61%|██████    | 4370/7218 [41:09<26:05,  1.82it/s]

 61%|██████    | 4371/7218 [41:10<33:05,  1.43i

13497 image downloaded
